In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
LINEAR = 6
DILUTION = 5

standard = ["A", "B", "C"]
experiment = ["D", "E", "F"]
compute_linear = {}
exp_data = {}
concentration = [5, 2.5, 1.25, 0.625, 0.3125, 0.15625]
concentration = concentration[::-1]
        
def promt_input(well_name):
    while True:
        x = input(f"{well_name}: ")
        try:
            x = float(x)
        except ValueError:
            continue
        if x > 0 and x < 1:
            break
    return x

def check_correctness(plate):
    while True:
        x = input("Any values you would like to change? ")
        if x.lower() in ["n", "no"]:
            break
        x = x.split()
        plate[x[0]] = float(x[1])
        print(f"{x[0]} changed to {x[1]}")
        
def number_experiments():
    while True:
        x = input("How many experiments are you performing? ")
        try: 
            x = int(x)
        except ValueError:
            continue
        break
    return x

def experiment_name(experiment_number):
    lst = []
    for i in range(experiment_number):
        x = input(f"Name of your experiment #{i+1}: ")
        lst.append(x)
    return lst

def print_experiment(lst):
    print("Your experiments are the following...")
    for i in range(len(lst)):
        print(f"#{i+1} - {lst[i]}") 
        
def display_plate(plate):
    print(str((format(plate["A1"], ".3f")) + " | " + str(format(plate["A2"], ".3f")) + " | " + str(format(plate["A3"], ".3f")) + " | " + str(format(plate["A4"], ".3f")) + " | " + str(format(plate["A5"], ".3f")) + " | " + str(format(plate["A6"], ".3f"))))
    print(str((format(plate["B1"], ".3f")) + " | " + str(format(plate["B2"], ".3f")) + " | " + str(format(plate["B3"], ".3f")) + " | " + str(format(plate["B4"], ".3f")) + " | " + str(format(plate["B5"], ".3f")) + " | " + str(format(plate["B6"], ".3f"))))
    print(str((format(plate["C1"], ".3f")) + " | " + str(format(plate["C2"], ".3f")) + " | " + str(format(plate["C3"], ".3f")) + " | " + str(format(plate["C4"], ".3f")) + " | " + str(format(plate["C5"], ".3f")) + " | " + str(format(plate["C6"], ".3f"))))
    for i in range(exp):
        print(lst[i])
        print("D" + str(i+1), format(plate["D" + str(i+1)], ".3f"))
        print("E" + str(i+1), format(plate["E" + str(i+1)], ".3f"))
        print("F" + str(i+1), format(plate["F" + str(i+1)], ".3f"))

# How many experiments one has?
exp = number_experiments()
lst = experiment_name(exp)
print_experiment(lst)

# Generate plate
plate = {}
for i in range(LINEAR):
    for j in standard:
        plate[j + str(i+1)] = 0
for i in range(exp):
    for j in experiment:
        plate[j + str(i+1)] = 0

for i in range(LINEAR):
    for j in standard:
        plate[j + str(i + 1)] = promt_input(j + str(i+1))
for i in range(exp):
    for j in experiment:
        plate[j + str(i + 1)] = promt_input(j + str(i + 1))

# Promt user if he/she wants to change any values
display_plate(plate)
check_correctness(plate)

# Calculate the average value for three measurments
for i in range(LINEAR):
    mean = 0
    for j in standard:
        mean += plate[j + str(i+1)]
    compute_linear[str(concentration[i])] = mean / 3
    

for i in range(exp):
    mean = 0
    for j in experiment:
        mean += plate[j + str(i+1)]
    exp_data[lst[i]] = mean / 3


How many experiments are you performing? 4
Name of your experiment #1: exp1
Name of your experiment #2: exp2
Name of your experiment #3: exp3
Name of your experiment #4: exp4
Your experiments are the following...
#1 - exp1
#2 - exp2
#3 - exp3
#4 - exp4
A1: 0,108
A1: 0.108
B1: 0.096
C1: 0.106
A2: 0.116
B2: 0.112
C2: 0.122
A3: 0.131
B3: 0.128
C3: 0.136
A4: 0.178
B4: 0.181
C4: 0.192
A5: 0.260
B5: 0.273
C5: 0.277
A6: 0.376
B6: 0.429
C6: 0.440
D1: 0.180
E1: 0.178
F1: 0.166
D2: 0.150
E2: 0.088
F2: 0.096
D3: 0.215
E3: 0.158
F3: 0.182
D4: 0.180
E4: 0.184
F4: 0.15
0.108 | 0.116 | 0.131 | 0.178 | 0.260 | 0.376
0.096 | 0.112 | 0.128 | 0.181 | 0.273 | 0.429
0.106 | 0.122 | 0.136 | 0.192 | 0.277 | 0.440
exp1
D1 0.180
E1 0.178
F1 0.166
exp2
D2 0.150
E2 0.088
F2 0.096
exp3
D3 0.215
E3 0.158
F3 0.182
exp4
D4 0.180
E4 0.184
F4 0.150
Any values you would like to change? F4 0.154
F4 changed to 0.154
Any values you would like to change? no


In [4]:
# Perform linear regression
X = []
y = []
i = 0
for conc, absorbance in compute_linear.items():
    X.append(absorbance)
    y.append(float(conc))

X = np.array(X).reshape(-1,1)
y = np.array(y).reshape(-1,1)
    
model = LinearRegression()
model.fit(X, y)

r_sq = model.score(X, y)

print("Coefficient of determination: ", r_sq)

print("intercept:", model.intercept_)
print("slope:", model.coef_)

Coefficient of determination:  0.9967100619945573
intercept: [-1.48942326]
slope: [[15.3894752]]


In [5]:
# Calculate protein concentration
b = float(model.intercept_[0])
a = float(model.coef_[0])

protein_concentrations = {}
for name, value in exp_data.items():
    protein_conc = a * value + b
    protein_concentrations[name] = protein_conc * 5
    print(name, "     -     ",  protein_concentrations[name])

exp1      -      5.993025367424297
exp2      -      1.1196915547815423
exp3      -      6.788148252644954
exp4      -      5.839130615446104


In [70]:
import xlsxwriter

workbook = xlsxwriter.Workbook("Expenses01.xlsx")
worksheet = workbook.add_worksheet()

# Put the names where needed
worksheet.write(0, 0, "Standards (mg/ml)")
worksheet.write(0, 1, "OD Standard Triplicate")
worksheet.write(0, 2, "Avg of Standards")
worksheet.write(0, 3, "Sample Name")
worksheet.write(0, 4, "OD Sample Triplicate")
worksheet.write(0, 5, "Avg OD of Samples")
worksheet.write(0, 7, "Protein concentration * dilution factor (mg/ml)")
worksheet.write(0, 8, "Volume of Sample")
worksheet.write(0, 9, "Volume of Loading Buffer")
worksheet.write(0, 10, "Volume of Buffer")
worksheet.write(0, 11, "Sample Name")

# Transfer the Absorbance measurment data
row = 1
col = 1
i = 0
cell_format = workbook.add_format()
cell_format01 = workbook.add_format()

cell_format01.set_align('center')
cell_format.set_num_format('0.000')
cell_format.set_align("center")

for name, absorbance in plate.items():
    if i == 18:
        col += 3
        row = 1
    worksheet.write(row, col, absorbance, cell_format)
    row += 1
    i += 1

# Increase the width of the columns
worksheet.set_column('A:L', 30)

# Create a format to use in the merged range.
merge_format = workbook.add_format({
    'align': 'center',
    'valign': 'vcenter',
    'num_format': '0.000'
})

# Merge cells in "Standards" column
for i in range(6):
    worksheet.merge_range("A"+str(i*3+2)+":A"+str(i*3+4), concentration[i], merge_format)

# Merge cells in "Avg OD of Standards"
i = 0
for absorbance in compute_linear.values():
    worksheet.merge_range("C"+str(i*3+2)+":C"+str(i*3+4), absorbance, merge_format)
    i += 1

# Merge cells in "Sample name"
for i in range(len(lst)):
    worksheet.merge_range("D"+str(i*3+2)+":D"+str(i*3+4), lst[i], merge_format)

# Merge cells in "Avg OD of Samples"
i = 0
for absorbance in exp_data.values():
    worksheet.merge_range("F"+str(i*3+2)+":F"+str(i*3+4), absorbance, merge_format)
    i += 1

# Write the formula and R^2
worksheet.write(0, 6, "Formula")
if b >= 0:
    worksheet.write(1, 6, f"y = {round(a, 4)}x + {round(b, 4)}")
else:
    worksheet.write(1, 6, f"y = {round(a, 4)}x {round(b, 4)}", cell_format01)

worksheet.write(3, 6, "Coefficient of Determination")
worksheet.write(4, 6, r_sq, cell_format01)

# Protein concentration 
i = 0
for pt_conc in protein_concentrations.values():
    worksheet.merge_range("H"+str(i*3+2)+":H"+str(i*3+4), pt_conc, merge_format)
    i += 1

workbook.close()

In [28]:
for x in exp_data.values():
    print(x)

0.17466666666666666
0.11133333333333333
0.18499999999999997
0.17266666666666666
